In [1]:
%matplotlib inline

In [2]:
import os
import torch

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [3]:
%load_ext autoreload
%autoreload 2

# load data

In [21]:
from transformer_translation.dataset import ParallelLanguageDataset
from torch.utils.data import DataLoader

In [44]:
data_path = r"/home/alex/data/nlp/agmir/transf_processed_data"
#data_path = 'transformer_translation/data/processed'

In [45]:
num_tokens = 2000
max_seq_length = 96
dataset = ParallelLanguageDataset(
    os.path.join(data_path, 'tags/set.pkl')#'en/train.pkl')#
    ,os.path.join(data_path, 'reports/set.pkl')#'fr/train.pkl')#
    ,num_tokens
    ,max_seq_length)
loader = DataLoader(dataset, batch_size=1, shuffle=True, num_workers=4, pin_memory=True)

# train

In [46]:
from transformer_translation.model import LanguageTransformer

from einops import rearrange
def gen_nopeek_mask(length):
    mask = rearrange(torch.triu(torch.ones(length, length)) == 1, 'h w -> w h')
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))

    return mask

In [47]:
vocab_size = 10000 + 4
d_model = 512
nhead = 8
num_encoder_layers = 6
num_decoder_layers = 6
dim_feedforward = 2048
pos_dropout = 0.1
trans_dropout = 0.1

model = LanguageTransformer(
    vocab_size, d_model, nhead, num_encoder_layers, num_decoder_layers, dim_feedforward,
    max_seq_length, pos_dropout, trans_dropout
).to('cuda')

In [48]:
    from transformer_translation.Optim import ScheduledOptim
    import torch.nn as nn
    from torch.optim import Adam
    
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_normal_(p)

    n_warmup_steps = 4000
    optim = ScheduledOptim(
        Adam(model.parameters(), betas=(0.9, 0.98), eps=1e-09),
        d_model, n_warmup_steps)

    criterion = nn.CrossEntropyLoss(ignore_index=0)

In [56]:
            src, src_key_padding_mask, tgt, tgt_key_padding_mask = next(iter(loader))

            src, src_key_padding_mask = src[0].to('cuda'), src_key_padding_mask[0].to('cuda')
            tgt, tgt_key_padding_mask = tgt[0].to('cuda'), tgt_key_padding_mask[0].to('cuda')

            memory_key_padding_mask = src_key_padding_mask.clone()
            tgt_inp, tgt_out = tgt[:, :-1], tgt[:, 1:]
            tgt_mask = gen_nopeek_mask(tgt_inp.shape[1]).to('cuda')
            
            outputs = model(src, tgt_inp, src_key_padding_mask, tgt_key_padding_mask[:, :-1], memory_key_padding_mask, tgt_mask)

In [58]:
outputs.shape

torch.Size([22, 94, 10004])

In [25]:
    print_every = 500
    model.train()

    lowest_val = 1e9
    val_losses = []
    total_step = 0
    
    for epoch in range(1):
        
        total_loss = 0

        for step, (src, src_key_padding_mask, tgt, tgt_key_padding_mask) in enumerate(iter(loader)):
            total_step += 1

            src, src_key_padding_mask = src[0].to('cuda'), src_key_padding_mask[0].to('cuda')
            tgt, tgt_key_padding_mask = tgt[0].to('cuda'), tgt_key_padding_mask[0].to('cuda')

            memory_key_padding_mask = src_key_padding_mask.clone()
            tgt_inp, tgt_out = tgt[:, :-1], tgt[:, 1:]
            tgt_mask = gen_nopeek_mask(tgt_inp.shape[1]).to('cuda')

            optim.zero_grad()
            outputs = model(src, tgt_inp, src_key_padding_mask, tgt_key_padding_mask[:, :-1], memory_key_padding_mask, tgt_mask)
            loss = criterion(rearrange(outputs, 'b t v -> (b t) v'), rearrange(tgt_out, 'b o -> (b o)'))

            loss.backward()
            optim.step_and_update_lr()

            total_loss += loss.item()
            if step % print_every == print_every - 1:
                print(f'Epoch [{epoch + 1} / {num_epochs}] \t Step [{step + 1} / {len(train_loader)}] \t '
                      f'Train Loss: {total_loss / print_every}')
                total_loss = 0

NameError: name 'gen_nopeek_mask' is not defined

In [14]:
from models import EncoderRNN, AttnDecoderRNN

In [15]:
from train_utils import trainIters

In [16]:
%%time
hidden_size = 256
max_length = 30
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, max_length, dropout_p=0.1).to(device)

plot_losses = trainIters(ds, encoder1, attn_decoder1, 100, max_length, input_lang, output_lang, print_every=10)

0m 1s (- 0m 10s) (10 10%) 7.0445
0m 2s (- 0m 9s) (20 20%) 5.8858
0m 3s (- 0m 8s) (30 30%) 6.0647
0m 4s (- 0m 6s) (40 40%) 5.3112
0m 4s (- 0m 4s) (50 50%) 5.5205
0m 5s (- 0m 3s) (60 60%) 4.6969
0m 6s (- 0m 2s) (70 70%) 5.2753
0m 7s (- 0m 1s) (80 80%) 4.4765
0m 8s (- 0m 0s) (90 90%) 4.9247
0m 9s (- 0m 0s) (100 100%) 4.4294
CPU times: user 10.2 s, sys: 316 ms, total: 10.5 s
Wall time: 10.6 s


# eval

In [21]:
from perf import get_av_bleu
from test_utils import evaluateAll, evaluateRandomly

In [27]:
%%time
pred_output = evaluateAll(ds, encoder1, attn_decoder1, max_length, input_lang, output_lang)

CPU times: user 1min 45s, sys: 1.23 s, total: 1min 46s
Wall time: 1min 46s


In [28]:
print('BLEU: {:.2%}'.format(
    get_av_bleu(pred_output)))

/home/alex/anaconda3/envs/bcic1/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


BLEU: 3.24%


# WIP plot learning curve

In [47]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)